In [6]:
# 最適化の近似がどの程度できているのか？

In [7]:
import sys
import os
import glob
import random
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import  open_json
import copy
from utils.util import calc_increase_c
from utils.versatility import calc_increase_v
from utils.util import calc_cw_score
from itertools import combinations

In [8]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [9]:
ver = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_versatility.json')
com = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json')

In [10]:
from utils.util import filter_basic_items, get_category

items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in filter_basic_items(json_dict["items"]):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [11]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [14]:
import importlib
import utils
importlib.reload(utils.util)
from utils.util import calc_cw_score
from utils.util import calc_increase_c
from utils.util import optimize_cw, scale_compatibility, scale_versatility

In [24]:
from utils.util import get_progress_percent

for i in range(100):
    dataset = {
        'tops':random.sample(items["tops"], 10),
        'bottoms':random.sample(items["bottoms"], 10),
        'shoes':random.sample(items["shoes"], 10),
    }

    # cwの初期化
    cw = {
        'tops':random.sample(dataset["tops"], T),
        'bottoms':random.sample(dataset["bottoms"], T),
        'shoes':random.sample(dataset["shoes"], T),
    }
    cw = optimize_cw(
        cw,
        dataset,
        com,
        ver,
    )
    c, v = calc_cw_score(cw, com, ver)
    c, v = scale_compatibility(c), scale_versatility(v)
    submoduler = c + v
    print(f'{i} fin submoduler', c, v)
    greedy_score = 0
    i = 0
    size = pow(10, 3) * pow(9, 3) * pow(8, 3) / (pow(3, 3) * pow(2, 3)) 
    for tops in list(combinations(dataset['tops'], T)):
        for bottoms in list(combinations(dataset['bottoms'], T)):
            for shoes in list(combinations(dataset['shoes'], T)):
                i += 1
                print(get_progress_percent(i, int(size)), end='')
                cw = {
                    'tops': tops,
                    'bottoms': bottoms,
                    'shoes': shoes
                }
                c, v = calc_cw_score(cw, com, ver)
                c, v = scale_compatibility(c), scale_versatility(v)
                greedy_score = max(greedy_score, c + v)
    print()
    print('greedy score: ', greedy_score)
    with open('C:/Users/yuuta/Documents/fashion/experiments/optimization/result/optim.txt', 'a') as f:
        f.write(f'{submoduler} {greedy_score}\n')

0 fin submoduler 1.9481572229480002 0
【*                   】

KeyboardInterrupt: 

In [ ]:
# compatibilityは、最小化しなきゃいけない値